# Training a digit classifier based on Chapter 4 of the fast.ai book

Chapter 4 gives an example for classifying 3s and 7s. Here I will train a model to classify all digits. 

In [255]:
!pip install fastai
from fastai.vision.all import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [256]:
!pip install fastbook
from fastbook import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1. Load data
Let's download the MNIST dataset.

In [257]:
path = untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/4'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/2'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/0')]

In [258]:
digit_paths = [(path/'training'/f'{o}').ls().sorted() for o in range(0,10)]
#Print a random 4
im4_path = digit_paths[4][1]
im4 = Image.open(im4_path)
im4

Let's turn the images into PyTorch tensors. 

In [259]:
digit_tensors = [[tensor(Image.open(o)) for o in digit_paths[i]] for i in range(0,10)]

Stack the list of digits into 10 separate tensors

In [260]:
stacked_digits = [torch.stack(digit_tensors[i]) for i in range(0,10)]

In [261]:
stacked_digits[4].size() 

torch.Size([5842, 28, 28])

To make sure that the pixel values are between 0 and 1 divide them by 255.

In [262]:
scaled_digits = [stacked_digits[i].float()/255 for i in range(0,10)]

In [263]:
scaled_digits[4].shape

torch.Size([5842, 28, 28])

### Set up training
1. Define the output, the training set, the test set and the validations set
2. Define the classification and the loss function
3. Define a metric for accuracy
4. Pick an example digit

In [264]:
#1. The training set
data_x = torch.cat([scaled_digits[i] for i in range(0,10)]).view(-1,28*28)
data_x.shape
#The outcomes
#Define the output as a vector of length 10 with 1 if the image is of that indices digit, 0 otherwise
tensor_id = torch.eye(10)
data_y = torch.cat([tensor_id[i].repeat(len(scaled_digits[i]),1) for i in range(0,10)])

dataset = list(zip(data_x, data_y))

train_len = int(len(dataset)*0.7)
test_len = int(len(dataset)*0.2)
valid_len = int(len(dataset)) - train_len - test_len

train, test, valid = torch.utils.data.random_split(dataset, lengths = [train_len, test_len, valid_len])
train_x = torch.stack([train[i][0] for i in range(0,len(train))])
train_y = torch.stack([train[i][1] for i in range(0,len(train))])

test_x = torch.stack([test[i][0] for i in range(0,len(test))])
test_y = torch.stack([test[i][1] for i in range(0,len(test))])

valid_x = torch.stack([valid[i][0] for i in range(0,valid_len)])
valid_y = torch.stack([valid[i][1] for i in range(0,valid_len)])

In [265]:
#2. The classification and the loss
#The image should be classified as the index with the highest element
#in the output, we can use the argmax() function for this

def loss(targets, predictions):
  return torch.sum((targets -predictions).square(), dim = 1).mean()

#Cross entropy loss
def cross_entropy_loss(targets, predictions):
  return F.cross_entropy(targets, predictions)
  #return torch.sum(targets*torch.log(predictions), dim = 1)


In [266]:
#3. Define accuracy
def batch_accuracy(xb, yb):
  correct_preds = (xb.argmax(dim = 1)) == (yb.argmax(dim = 1))
  return correct_preds.float().mean()

In [267]:
#4. Pick an example digit
example_digit = scaled_digits[4][1].view(-1,28*28)
example_digit.shape
example_dig_shaped =  scaled_digits[4][1]

## 2. Build a neural net model all the way from matrices
For the sake of simplicity I will use two layers with a sigmoid non-linarity. 

In [268]:
#1. Initialize the parameters
w = torch.randn(28*28,10).requires_grad_()
b = torch.randn(10).requires_grad_()

In [269]:
#2. Calculate the predictions

#Define the linear function (layer) 
def linear_layer(inp,w,b):
    return (inp@w) + b

#Prediction example
pred_vector = linear_layer(example_digit, w,b)
pred = pred_vector.argmax() 

#Calculate the prediction for all of train_x
predictions_train_x = torch.stack([linear_layer(dig, w,b) for dig in train_x])

In [270]:
#TODO: 3. Calculate the loss
loss_1 = loss(predictions_train_x, train_y)  
loss_1

tensor(996.7764, grad_fn=<MeanBackward0>)

In [271]:
#4. Calculate the gradients
loss_1.backward()
w.grad
b.grad

tensor([ 13.7707,  13.6216,  12.7869,  18.3310,  12.4007, -13.6310,  -5.3897, -21.7068,   2.5706,  -4.1771])

In [272]:
#5. Step the weights
#Pick learning rate
lr = 1e-3
w.data -= lr*w.grad.data
b.data -= lr*b.grad.data
w.grad = None
b.grad = None

In [273]:
improved_predictions_train_x = torch.stack([linear_layer(dig, w,b) for dig in train_x])

In [274]:
loss_2 = loss(improved_predictions_train_x, train_y)
loss_2

tensor(922.9205, grad_fn=<MeanBackward0>)

Now let's put all of this together and train a model using matrix calculations

In [275]:
#Let's put all of this together
def initialize():
    w = torch.randn(28*28,10).requires_grad_()
    b = torch.randn(10).requires_grad_()
    return w,b

def step(w,b, lr):
    predictions_train_x = torch.stack([linear_layer(dig, w,b) for dig in train_x])
    loss_d = loss(predictions_train_x, train_y)
    print(loss_d)
    loss_d.backward()
    w.data -= lr*w.grad.data
    w.grad = None
    b.data -= lr*b.grad.data
    b.grad = None


In [276]:
w,b = initialize()
lr = 1e-3
for i in range(0,10):
    step(w,b,lr)

example_prediction = linear_layer(example_digit, w,b)
print(example_prediction)

tensor(1109.4843, grad_fn=<MeanBackward0>)
tensor(1010.5391, grad_fn=<MeanBackward0>)
tensor(926.0790, grad_fn=<MeanBackward0>)
tensor(853.9323, grad_fn=<MeanBackward0>)
tensor(792.2540, grad_fn=<MeanBackward0>)
tensor(739.4761, grad_fn=<MeanBackward0>)
tensor(694.2659, grad_fn=<MeanBackward0>)
tensor(655.4906, grad_fn=<MeanBackward0>)
tensor(622.1879, grad_fn=<MeanBackward0>)
tensor(593.5399, grad_fn=<MeanBackward0>)
tensor([[ 12.0657,   2.7091,   4.9355,  -7.2724,  11.4998,  -3.0284,  -0.5546,   8.0484, -14.3384,  -9.1840]], grad_fn=<AddBackward0>)


In [277]:
#Add a softmax function to normalize the output vetor. This is to obtain 'probabilities'
def step_sig(w,b, lr):
    predictions_train_x = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in train_x])
    loss_d = loss(predictions_train_x, train_y)
    print(loss_d)
    loss_d.backward()
    w.data -= lr*w.grad.data
    w.grad = None
    b.data -= lr*b.grad.data
    b.grad = None


In [278]:
#import pytorch as nn
#torch.softmax(example_prediction, dim = 10)
torch.nn.functional.softmax(example_prediction, dim = 1)

tensor([[6.3021e-01, 5.4449e-05, 5.0455e-04, 2.5181e-09, 3.5788e-01, 1.7547e-07, 2.0824e-06, 1.1345e-02, 2.1494e-12, 3.7229e-10]], grad_fn=<SoftmaxBackward0>)

In [287]:
w,b = initialize()
lr = 0.1
for i in range(0,10):
    step_sig(w,b,lr)

example_prediction = torch.nn.functional.softmax(linear_layer(example_digit, w,b))
print(example_prediction)
print(example_prediction.argmax())

<ipython-input-277-d77ed0369ab9>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions_train_x = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in train_x])


tensor(1.6405, grad_fn=<MeanBackward0>)
tensor(1.6400, grad_fn=<MeanBackward0>)
tensor(1.6395, grad_fn=<MeanBackward0>)
tensor(1.6389, grad_fn=<MeanBackward0>)
tensor(1.6384, grad_fn=<MeanBackward0>)
tensor(1.6379, grad_fn=<MeanBackward0>)
tensor(1.6373, grad_fn=<MeanBackward0>)
tensor(1.6368, grad_fn=<MeanBackward0>)
tensor(1.6363, grad_fn=<MeanBackward0>)
tensor(1.6357, grad_fn=<MeanBackward0>)
tensor([[9.8051e-09, 2.1594e-07, 7.1730e-03, 6.1965e-10, 9.8852e-01, 1.6105e-07, 1.0146e-10, 7.6818e-14, 4.3062e-03, 2.2313e-09]], grad_fn=<SoftmaxBackward0>)
tensor(4)


<ipython-input-287-1715ed3df380>:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  example_prediction = torch.nn.functional.softmax(linear_layer(example_digit, w,b))


In [280]:
#Test 
predictions_test = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in test_x])
loss_test = loss(predictions_test, test_y)
loss_test

<ipython-input-280-087942e95b11>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions_test = torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in test_x])


tensor(1.6604, grad_fn=<MeanBackward0>)

In [301]:
#Accuracy on the validation dataset
predictions_valid =  torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in valid_x])
valid_accuracy = batch_accuracy(predictions_valid, valid_y)
valid_accuracy

<ipython-input-301-f9ef14669537>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions_valid =  torch.stack([torch.nn.functional.softmax(linear_layer(dig, w,b)) for dig in valid_x])


tensor(0.0912)

Not surprisingly, the accuracy on the validation set of this model is very low. The model consists of one linear layer with 10 neurons, which is too  little for this task. Also, here I am not using CrossEntropyLoss yet, but an intuitive extension of the loss for classifying two digits in Ch4 of the fastai book. 

## 3. Build a neural net with PyTorch and fastai

In [296]:
#Load the data
dl = DataLoader(train, batch_size = 256)
valid_dl = DataLoader(test, batch_size=256)
final_valid_dl = DataLoader(valid, batch_size = 256)
dls = DataLoaders(dl,valid_dl)

In [297]:
#Set up the architecture
neural_net  = nn.Sequential(
    nn.Linear(28*28,60),
    nn.ReLU(),
    nn.Linear(60,10),
    nn.Softmax(dim=1)
)

In [298]:
learn = Learner(dls, neural_net, opt_func = SGD, loss_func = cross_entropy_loss, metrics = batch_accuracy)

In [299]:
learn.fit(10, lr = 0.01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,2.301280,2.300797,0.121333,00:01
1,2.298327,2.297666,0.165000,00:01
2,2.294736,2.293916,0.186250,00:01
3,2.290127,2.289042,0.195333,00:01
4,2.283616,2.282061,0.188500,00:01
5,2.273690,2.271416,0.171917,00:01
6,2.258690,2.255920,0.172000,00:01
7,2.241222,2.239329,0.220000,00:01
8,2.224674,2.223378,0.280000,00:01
9,2.207715,2.206279,0.315417,00:01


In [291]:
#Classifying digits in hte validation set
valid_preds, _ = learn.get_preds(dl = final_valid_dl)
valid_preds_class = valid_preds.argmax(dim = 1)

validation_accuracy = batch_accuracy(valid_preds, valid_y)
validation_accuracy

tensor(0.4640)

Of course, the accuracy of this super simple neural net model is not very high. The goal of this notebook was to practice the steps of training a model both from scratch and using PyTorch, as well as to create a working base version of an MNIST classifier. 
